<a href="https://colab.research.google.com/github/Anshuman-02905/Custom_Satellite_Scripts/blob/main/GetSentinel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''gdalinput = '/content/ADASA_UG_TO_OC.tif'
gdaloutput = '/content/ADASA_UG_TO_OC.png'
options_list = ['-ot Byte','-of PNG','-scale 0 0.3 0 255'] 
options_string = " ".join(options_list)    
gdal.Translate(gdaloutput, gdalinput, options=options_string)'''

In [34]:
'''import os
import glob

files = glob.glob('/content/drive/MyDrive/Images/*')
for f in files:
    os.remove(f)'''

'''

In [ ]:
!pip install geemap
!pip install geopandas
!apt-get update
!apt-get install libgdal-dev -y
!apt-get install python-gdal -y
!apt-get install python-numpy python-scipy -y

In [13]:
import geemap
import os
import glob
import gdal ## fingers crossed!
import os
import ee

In [15]:
Map=geemap.Map()

In [39]:
missing=[]
Shape_path="/content/drive/MyDrive/mines"
mine_path="/content/drive/MyDrive/Sentinel_raw/"
image_path="/content/drive/MyDrive/Images/"

In [45]:

def download_image(mine_shape,mine_input,image_output):
  dataset = ee.ImageCollection('COPERNICUS/S2_SR').select(['B4', 'B3', 'B2']) \
                  .filterDate('2017-01-01', '2020-01-30') \
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',5)) \
                  .filterBounds(mine_shape)

  visualization = {
  'min': 0.0,
  'max': 0.3,
  'bands': ['B4', 'B3', 'B2'],
  }

  #Map.centerObject(dataset, 12)
  #Map.addLayer(dataset.mean().divide(10000).clip(mine), visualization, 'RGB')

  image = dataset.mean().divide(10000).select(['B4', 'B3', 'B2']).unmask()
  geemap.ee_export_image(image, filename=mine_input, scale=10, region=mine.geometry(), file_per_band=False)

  options_list = ['-ot Byte','-of PNG','-scale 0 0.3 0 255'] 
  options_string = " ".join(options_list) 
  if os.path.exists(mine_input):   
    gdal.Translate(image_output, mine_input, options=options_string)
    rem_path=image_output+".aux.xml"
    os.remove(rem_path)
  else:
    missing.append(mine_name)


In [ ]:
for path, subdirs, files in os.walk(Shape_path):
  #print(files)
  for item in files:
    if(item.split('.')[-1]=='shp' and item.split('.')[0][-3:]!="gcs"):
      #print(path+"/"+item)
      mine_shp = path+"/"+item
      mine = geemap.shp_to_ee(mine_shp)
      mine_name=item.split('.')[0]
      mine_input=mine_path+mine_name+".tif"
      image_output=image_path+mine_name+".png"
      rem_path=image_output+".aux.xml"
      #print(rem_path)
      print("Downloading the image of mine: "+mine_name)
      download_image(mine,mine_input,image_output)
      
    